In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'numpy'

In [3]:
!pip install numpy

zsh:1: /opt/homebrew/bin/pip: bad interpreter: /opt/homebrew/opt/python35/bin/python3.5: no such file or directory


In [ ]:
!ls